<a href="https://colab.research.google.com/github/J3SSUS/peruvian-football-news-generator/blob/main/gpt2/gpt2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT2 - Peruvian Football News Article Generator

In [ ]:
!pip install torch

In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-quzy7q8f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-quzy7q8f
  Resolved https://github.com/huggingface/transformers to commit 2fc33ebead50383f7707b17f0e2a178d86347d10
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=7995423 sha256=f3d5190c42e4bfe54369ce1fcdbfd78221437c1d134e7db442647dcdf398b5c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-qpun

In [ ]:
import os
import time
import datetime

import numpy as np
import random
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)

if torch.cuda.is_available():
    print("Usar GPU")
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    # Tell pytorch to run this model on the GPU.
    device = torch.device("cuda")
    batch_size = 3

else:
    print("usar CPU")
    device = torch.device("cpu")
    batch_size = 1

Usar GPU


In [ ]:
# Load the GPT tokenizer.

tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

model = AutoModelForCausalLM.from_pretrained("flax-community/gpt-2-spanish")

# control_code = "ibai"

# special_tokens_dict = {
#          "additional_special_tokens": ['f"<|{control_code}|>"'],
# }
# num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
unk_tok_emb = model.transformer.wte.weight.data[tokenizer.unk_token_id, :]
# for i in range(num_added_toks):
#         model.transformer.wte.weight.data[-(i+1), :] = unk_tok_emb

In [ ]:
class GPT2Dataset(Dataset):

    def __init__(self, tokenizer, archivo_texto = 'all.txt', max_length=768):

        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []

        print('loading text...')
        sentences = open(archivo_texto, 'r', encoding="utf-8").read().lower().split('\n')
        print('qty:',len(sentences))

        for row in tqdm(sentences):
            encodings_dict = tokenizer('<|startoftext|>'+ row + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = GPT2Dataset(tokenizer, archivo_texto="/content/drive/MyDrive/nlp/project-nlp-football/gtp2/gtp2_peruvian_football_news.txt", max_length=768)

# Split into training and validation sets
train_size = int(0.99 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

loading text...
qty: 595


100%|██████████| 595/595 [00:00<00:00, 981.07it/s]

  589 training samples
    6 validation samples


In [ ]:
# some parameters to train
epochs = 20
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
# this produces sample output every x steps
sample_every = 500
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )
# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))


## Fine-tuning GPT2

In [ ]:
total_t0 = time.time()

model = model.to(device)

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(  b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,
                                    top_k=50,
                                    max_length = 200,
                                    top_p=0.95,
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

            model.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    t0 = time.time()

    total_eval_loss = 0
    nb_eval_steps = 0

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
#Average training loss: 0.28
#Training epoch took: 1:23:32 mode_save2 va bastante bien, solo 1 epoch


======== Epoch 1 / 20 ========
Training...

  Average training loss: 1.43
  Training epoch took: 0:02:18

======== Epoch 2 / 20 ========
Training...

  Average training loss: 0.76
  Training epoch took: 0:02:22

======== Epoch 3 / 20 ========
Training...

  Average training loss: 0.67
  Training epoch took: 0:02:22

======== Epoch 4 / 20 ========
Training...

  Average training loss: 0.60
  Training epoch took: 0:02:22

======== Epoch 5 / 20 ========
Training...

  Average training loss: 0.52
  Training epoch took: 0:02:22

======== Epoch 6 / 20 ========
Training...

  Average training loss: 0.44
  Training epoch took: 0:02:22

======== Epoch 7 / 20 ========
Training...

  Average training loss: 0.36
  Training epoch took: 0:02:22

======== Epoch 8 / 20 ========
Training...

  Average training loss: 0.27
  Training epoch took: 0:02:22

======== Epoch 9 / 20 ========
Training...

  Average training loss: 0.19
  Training epoch took: 0:02:22

======== Epoch 10 / 20 ========
Training...



In [ ]:
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
output_dir = '/content/drive/MyDrive/nlp/project-nlp-football/gtp2/model'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to /content/drive/MyDrive/nlp/project-nlp-football/gtp2/model


('/content/drive/MyDrive/nlp/project-nlp-football/gtp2/model/tokenizer_config.json',
 '/content/drive/MyDrive/nlp/project-nlp-football/gtp2/model/special_tokens_map.json',
 '/content/drive/MyDrive/nlp/project-nlp-football/gtp2/model/vocab.json',
 '/content/drive/MyDrive/nlp/project-nlp-football/gtp2/model/merges.txt',
 '/content/drive/MyDrive/nlp/project-nlp-football/gtp2/model/added_tokens.json',
 '/content/drive/MyDrive/nlp/project-nlp-football/gtp2/model/tokenizer.json')

## Inference

### 1 epoch

In [ ]:
model.eval()
prompt = "<|startoftext|>" + "Universitario ganó la final"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
sample_outputs = model.generate(
                                generated,
                                num_return_sequences=3,
                                max_length = 300,
                                do_sample=True,
                                top_k=50,
                                top_p=0.95
                                )
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}nn".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Universitario ganó la final del torneo Clausura tras vencer al San José 1 por 3. Los celestes, dirigidos por la dupla entre el argentino Miguel Ponce y el uruguayo Miguel Ángel Álvarez, jugaron una final más y, para este momento, tendrán que ganar la segunda categoría de la institución para volver a ser el primero en disputar este torneo.
Universitario consiguió la victoria por 4-3 ante San José y la final, en un partido muy duro. La vuelta a la final será este miércoles, desde las 20:00 horas de Perú en el estadio Carlos Ugalde y será transmitida por DIRECTV Sports.
Universitario logró así una merecida hazaña tras un largo torneo donde se coronó campeón por segunda vez consecutiva. Además, el técnico brasileño por lo que resta por jugar, los torneos clausura 2013 y 2014, para no perder la posibilidad de acceder a la máxima categoría del fútbol peruano, pero por por lo que resta por disputar. La final, el miércoles, a las 20:30 horas de Perú, enfrentará a Deportivo Pasto vs Universi

### 20 epochs

In [ ]:
model.eval()
prompt = "<|startoftext|>" + "Universitario ganó la final:"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
sample_outputs = model.generate(
                                generated,
                                num_return_sequences=3,
                                max_length = 300,
                                do_sample=True,
                                top_k=50,
                                top_p=0.95
                                )
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}nn".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Universitario ganó la final: hasta la fecha, los cremas han perdido la final de la liga 1 2023 ante Eibar por el título en el monumental de la victoria de los merengues sobre el juego más vistoso y emocionante hasta la fecha. universitario campeón 2024. universitario subcampeón 2023. 909 minutos 279 segundos 563 puntos impresionante el rendimiento de los cremas en la última fecha habían roto todas las marcas de la liga 1, pero esta vez ellos se quedaron detrás con 103 minutos de juego ganado por los embajadores 2026. para recordar este partido inolvidable crema campeón 2023. para recordar este final emocionante y conmovedoramente hermoso. para cerrar la jornada con el emocionante encuentro entre los cremas y united que terminó con un emocionante empate a uno para los seguidores denn
1: Universitario ganó la final: hasta la fecha, la Udenar tiene la copa libertadores del américa, sumándole la diferencia de goles, usted ha anotado uno más que nosotros?, fue la típica pregunta que hací

In [ ]:
model.eval()
prompt = "<|startoftext|>" + "Universitario ganó y se pone primero en el torneo Clausura"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
sample_outputs = model.generate(
                                generated,
                                num_return_sequences=3,
                                max_length = 300,
                                do_sample=True,
                                top_k=50,
                                top_p=0.95
                                )
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}nn".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Universitario ganó y se pone primero en el torneo Clausura: hasta el momento, es todo, fue un gran torneo para los cremas se llevaron el título. que que y esta madrugada utc ganó por 4-3 tamboras., fue un gran torneo para los blanquiazules se lo llevaron. no. no. no. no., hasta el momento, es todo del momento para los blanquiazules es un gran torneo para los blanquiazules han logrado el puntaje perfecto en promedio de golnn
1: Universitario ganó y se pone primero en el torneo Clausura: hasta que se niegue, será que el torneo clausura se disputará en el estadio monumental. válido por la fecha 18 de la segunda categoría del fútbol peruano. el gol de la victoria de universitario de deportes sobre alianza lima, correspondiente a la decimocuarta jornada. como se recuerda, un encuentro disputado por la fecha anterior, donde los cremas empataron 1 a 1, con un tanto del defensor de la crema, mónaco josé olaya. sin embargo, en el torneo clausura, los cremas empataron 1-1 con adt, en matute. 

In [ ]:
model.eval()
prompt = "<|startoftext|>" + "El torneo Clausura se puso interesante"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
sample_outputs = model.generate(
                                generated,
                                num_return_sequences=3,
                                max_length = 300,
                                do_sample=True,
                                top_k=50,
                                top_p=0.95
                                )
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}nn".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: El torneo Clausura se puso interesante con la presentación de Alianza Lima sacó un gran triunfo por 2-0 ante Unión Comercio en el estadio Alejandro Villanueva. los blanquiazules descontaron un par de minutos a un rival que intentó pero no pudo. luego del tanto de penal marcado por el argentino de la selección peruana, el partido se volvió loco y los hinchas locales salieron a respaldar al equipo. tras el encuentro, el entrenador uruguayo de la blanquirroja, julinho, se refirió al gol del triunfo y la reacción de su equipo. el equipo intentó siempre, pero no siempre gana. en el primer tiempo, el equipo visitante se hizo fuerte, pero no creó problemas. en el segundo tiempo, casi hizo el gol de la tranquilidad, solo lo intentaron en una ocasión y no lograron anotar. en el segundo tiempo, casi hizo el gol de la tranquilidad. en un contraataque, barreto anota el primer gol del 2-0 y todo fue emoción en el estadio. barreto mandó la tirolesa para los locales que fue todo un éxito. luego de

In [ ]:
model.eval()
prompt = "<|startoftext|>" + "Universitario ganó de visita"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
sample_outputs = model.generate(
                                generated,
                                num_return_sequences=3,
                                max_length = 300,
                                do_sample=True,
                                top_k=50,
                                top_p=0.95
                                )
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}nn".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Universitario ganó de visita 2-0 a Unión Comercio y se convirtió en el primer grande de la historia del fútbol peruano, informó el club dirigido por Gustavo Valdenegro. Kenji Kaisaro, Darwin Gandac y Yoshimar Yotún serán los autores del tanto que le dan la victoria a los cremas. Jeison Lizano, Darren Carrilo y Jeison Aguilera son los otros dos jugadores que anotaron de penal. Universitario ganó 2-0 a Unión Comercio y se convirtió en el primer grande de la historia del fútbol peruano, informó el club. Why Not Rosario? Universitario ganó 2-0 a Unión Comercio y se convirtió en el primer grande de la historia del fútbol peruano, informó el club. Gustavo Jalkhac y Jeison Aguilera anotaron de cabeza el primero e igualaron las acciones. Universitario gana 1-0 a Unión Comercio y es el primer grande de la historia del fútbol peruano, informó el club. Kenji Kaisaro, Darwin Gandac y Yoshimar Yotún son los otros dos jugadores que anotaron de cabeza en el marcador. Universitario ganó 2-0 a Unión